> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [ ]:
import pandas as pd

# Caminho local do CSV (assumindo que você já fez o download)
csv_path = "salaries.csv"

# Se o arquivo não existir localmente, tenta baixar direto do GitHub (modo fallback)
try:
    df = pd.read_csv(csv_path)
except FileNotFoundError:
    csv_url = "https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/salaries.csv"
    df = pd.read_csv(csv_url)

# Q1) Conhecendo os dados
print("Primeiras linhas do DataFrame:")
display(df.head())

print("\nInformações gerais do DataFrame:")
display(df.info())

print("\nDescrição estatística (colunas numéricas):")
display(df.describe())

#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [ ]:
import re
from enum import Enum as PyEnum

from sqlalchemy import Column, Integer, Float, String, Date, Enum as SAEnum
from sqlalchemy.orm import declarative_base

# Usaremos o DataFrame df carregado na Q1 para descobrir os valores únicos
df.head()

Base = declarative_base()

def criar_enum_dinamico(nome_enum: str, valores):
    """
    Cria uma Enum Python dinamicamente a partir de uma lista de valores,
    garantindo que os nomes dos membros sejam identificadores válidos.
    """
    valores_unicos = sorted({str(v) for v in valores if pd.notna(v)})
    membros = {}
    for valor in valores_unicos:
        # Cria um nome de membro válido (sem espaços, sem caracteres especiais, não iniciando por número)
        chave = re.sub(r"\W|^(?=\d)", "_", valor).upper()
        membros[chave] = valor
    return PyEnum(nome_enum, membros)

# Enums dinâmicos com base nos valores do CSV
SexEnum = criar_enum_dinamico("SexEnum", df["SEX"].unique())
DesignationEnum = criar_enum_dinamico("DesignationEnum", df["DESIGNATION"].unique())
UnitEnum = criar_enum_dinamico("UnitEnum", df["UNIT"].unique())

from datetime import datetime

def parse_date(date_str):
    """
    Converte datas no formato MM/DD/AAAA para datetime.date.
    Se a leitura falhar ou o valor for nulo, retorna None.
    """
    if pd.isna(date_str):
        return None
    if isinstance(date_str, datetime):
        return date_str.date()
    try:
        return datetime.strptime(str(date_str), "%m/%d/%Y").date()
    except Exception:
        return None

class SalaryRecord(Base):
    """Modelo ORM que representa a tabela de salários."""

    __tablename__ = "salaries"

    id = Column(Integer, primary_key=True, autoincrement=True)

    first_name = Column("FIRST NAME", String, nullable=False)
    last_name = Column("LAST NAME", String, nullable=False)

    sex = Column("SEX", SAEnum(SexEnum, name="sex_enum"), nullable=False)

    doj = Column("DOJ", Date, nullable=True)
    current_date = Column("CURRENT DATE", Date, nullable=True)

    designation = Column(
        "DESIGNATION", SAEnum(DesignationEnum, name="designation_enum"), nullable=False
    )

    age = Column("AGE", Integer, nullable=True)
    salary = Column("SALARY", Float, nullable=False)

    unit = Column("UNIT", SAEnum(UnitEnum, name="unit_enum"), nullable=False)

    leaves_used = Column("LEAVES USED", Integer, nullable=True)
    leaves_remaining = Column("LEAVES REMAINING", Integer, nullable=True)
    ratings = Column("RATINGS", Float, nullable=True)
    past_exp = Column("PAST EXP", Float, nullable=True)

    def __repr__(self):
        return (
            f"<SalaryRecord(id={self.id}, "
            f"name={self.first_name} {self.last_name}, "
            f"designation={self.designation}, salary={self.salary})>"
        )

SalaryRecord.__table__

#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [ ]:
from sqlalchemy import create_engine

# Q3) Criando conexão com banco SQLite chamado "salarios"
engine = create_engine("sqlite:///salarios.db", echo=False, future=True)
engine

#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [ ]:
# Q4) Criando as tabelas no banco "salarios"
Base.metadata.create_all(engine)
Base.metadata.tables

#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [ ]:
# Q5) Populando o banco com os dados do CSV usando pandas.to_sql

# Garante que as colunas de data estejam no tipo correto
if "DOJ" in df.columns:
    df["DOJ"] = df["DOJ"].apply(parse_date)

if "CURRENT DATE" in df.columns:
    df["CURRENT DATE"] = df["CURRENT DATE"].apply(parse_date)

# Insere os dados na tabela já criada pelo ORM
# if_exists="append" garante que a tabela não será dropada/recriada
linhas_antes = 0
with engine.connect() as conn:
    linhas_antes = conn.execute(
        "SELECT COUNT(*) FROM salaries"
    ).scalar()

df.to_sql(
    name=SalaryRecord.__tablename__,
    con=engine,
    if_exists="append",
    index=False,
)

with engine.connect() as conn:
    linhas_depois = conn.execute(
        "SELECT COUNT(*) FROM salaries"
    ).scalar()

print(f"Linhas antes da inserção: {linhas_antes}")
print(f"Linhas depois da inserção: {linhas_depois}")
print(f"Total inserido nesta execução: {linhas_depois - linhas_antes}")

#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [ ]:
from sqlalchemy import text

# Q6-A) SQL "puro" via engine.connect()

query_sql = """
SELECT
    "DESIGNATION" AS designation,
    MIN("SALARY" / 12.0) AS min_monthly_salary,
    MAX("SALARY" / 12.0) AS max_monthly_salary,
    AVG("SALARY" / 12.0) AS avg_monthly_salary
FROM salaries
GROUP BY "DESIGNATION"
ORDER BY avg_monthly_salary DESC;
"""

with engine.connect() as conn:
    result = conn.execute(text(query_sql))
    for row in result:
        print(
            f"Designation={row.designation:25s} | "
            f"Min={row.min_monthly_salary:10.2f} | "
            f"Max={row.max_monthly_salary:10.2f} | "
            f"Avg={row.avg_monthly_salary:10.2f}"
        )

In [ ]:
# Q6-B) Mesma query SQL, agora usando pandas.read_sql_query

with engine.connect() as conn:
    df_group = pd.read_sql_query(query_sql, conn)

df_group

In [ ]:
from sqlalchemy import select, func
from sqlalchemy.orm import Session

# Q6-C) Usando ORM (select + Session)

stmt = (
    select(
        SalaryRecord.designation.label("designation"),
        (func.min(SalaryRecord.salary / 12.0)).label("min_monthly_salary"),
        (func.max(SalaryRecord.salary / 12.0)).label("max_monthly_salary"),
        (func.avg(SalaryRecord.salary / 12.0)).label("avg_monthly_salary"),
    )
    .group_by(SalaryRecord.designation)
    .order_by(func.avg(SalaryRecord.salary / 12.0).desc())
)

with Session(engine) as session:
    result = session.execute(stmt).all()

for row in result:
    print(
        f"Designation={row.designation:25s} | "
        f"Min={row.min_monthly_salary:10.2f} | "
        f"Max={row.max_monthly_salary:10.2f} | "
        f"Avg={row.avg_monthly_salary:10.2f}"
    )